In [2]:
%load_ext autoreload
%autoreload 2



In [3]:
import joblib

In [4]:
import pandas as pd
from utils.preprocess import treat_dataset_pandas_init, build_preprocessing_pipeline,INIT_NUMERICAL_COLS, numerical_features

from utils.metrics import rmsle_metric

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

from xgboost import XGBRegressor
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import root_mean_squared_log_error
from sklearn.ensemble import StackingRegressor, HistGradientBoostingRegressor
import numpy as np
import xgboost as xgb
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsRegressor

from catboost import CatBoostRegressor
from utils.categoricals import CategoricalToStringTransformer, CategoricalEncoder, CatogoricalUnknownTransformer



In [20]:
train_data = pd.read_csv("playground-series-s4e12/train.csv")#.sample(frac=0.1)
test_data = pd.read_csv("playground-series-s4e12/test.csv")

treated_dataset = treat_dataset_pandas_init(train_data, process_as_category=True)
treated_test_dataset = treat_dataset_pandas_init(test_data, process_as_category=True)


#non_loged_train, non_loged_test = joblib.load("cat_non_loged.pkl")

#treated_dataset['non_log_oof_prediction'] = non_loged_train

X_train = treated_dataset.drop(columns=["Premium Amount"])
y_train = treated_dataset["Premium Amount"]

treated_test_dataset = treated_test_dataset[X_train.columns]


In [6]:
from sklearn.linear_model import BayesianRidge


In [7]:
X_train

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,...,Income to Dependents Ratio,Income_per_Dependent,CreditScore_InsuranceDuration,Health_Risk_Score,Credit_Health_Score,Health_Age_Interaction,Claims v Duration,Health vs Claims,Cat Credit Score,Int Credit Score
0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,...,5024.5,5024.5,1860.0,3.870062,8406.738970,429.376453,0.400000,7.532920,372.0,372.0
1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,Comprehensive,...,7919.5,7919.5,1388.0,4.221513,10805.393307,607.219509,0.500000,7.784865,694.0,694.0
2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,...,6400.5,6400.5,NaN,2.641123,NaN,1085.083634,0.333333,23.588775,<NA>,NaN
3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,Basic,...,47285.0,47285.0,367.0,4.453093,4014.298906,229.701027,1.000000,5.469072,367.0,367.0
4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,Premium,...,19825.5,19825.5,2392.0,3.981195,12184.903989,427.897966,0.000000,20.376094,598.0,598.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,36.0,Female,27316.0,Married,0.0,Master's,Unemployed,13.772907,Urban,Premium,...,27316.0,27316.0,1116.0,4.311355,5123.521323,495.824644,NaN,NaN,372.0,372.0
1199996,54.0,Male,35786.0,Divorced,NaN,Master's,Self-Employed,11.483482,Rural,Comprehensive,...,35786.0,NaN,2388.0,4.425826,6855.638903,620.108041,NaN,NaN,597.0,597.0
1199997,19.0,Male,51884.0,Divorced,0.0,Master's,NaN,14.724469,Suburban,Basic,...,51884.0,51884.0,NaN,4.263777,NaN,279.764902,0.000000,14.724469,<NA>,NaN
1199998,55.0,Male,NaN,Single,1.0,PhD,NaN,18.547381,Suburban,Premium,...,NaN,NaN,1628.0,4.072631,7548.784101,1020.105960,0.250000,9.273691,407.0,407.0


In [8]:
from datetime import datetime
from sklearn.model_selection import KFold


In [9]:
# Define a 60-40 split
splitter = ShuffleSplit(n_splits=5, test_size=0.2, random_state=1)
splitter = KFold(n_splits=5, shuffle=True, random_state=42)

results = []
models = []
for train_idx, val_idx in splitter.split(X_train):
    print("Starting")
    # Prepare data for LightGBM
    X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    {'learning_rate': 0.4290296903723802, 'n_estimators': 865, 'max_depth': 4, 'min_child_weight': 1, 'subsample': 0.8352284080073455, 'colsample_bytree': 0.7100615955636335, 'gamma': 2.8086205574716985, 'lambda': 2.9562811725383225, 'alpha': 9.51949783566794}
    
    full_pipeline = Pipeline([
        ("model",xgb.XGBRegressor(
            learning_rate=0.4290296903723802,
            n_estimators=187,
            max_depth=4,
            min_child_weight=1,
            subsample=0.8352284080073455,
            colsample_bytree=0.7100615955636335,
            gamma=2.8086205574716985,
            reg_lambda=2.9562811725383225,
            reg_alpha=9.51949783566794,
            enable_categorical=True,
            tree_method="hist",
            objective="reg:squarederror",
            device="cuda"
        ))
    ])
    full_pipeline.fit(X_train_fold, y_train_fold)
    models.append(full_pipeline)
    
    train_predictions = np.clip(full_pipeline.predict(X_train_fold), 20, 5000)
    val_predictions = np.clip(full_pipeline.predict(X_val_fold), 20, 5000)
    results.append({
        "oof_predictions": {
            "index": val_idx,
            "predictions": val_predictions,
        },
        "test_predictions": {
            "predictions": np.clip(full_pipeline.predict(treated_test_dataset), 20, 5000),
        }
    })

    train_score = root_mean_squared_log_error(train_predictions, y_train_fold)
    val_score = root_mean_squared_log_error(val_predictions, y_val_fold)

    print(f"Train: {train_score}, Test: {val_score}")

Starting


d:\Development\insurance\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [01:02:32] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Train: 1.1313341554478178, Test: 1.1409681885080667
Starting
Train: 1.1334130017937218, Test: 1.1405699811180052
Starting
Train: 1.1318561732730583, Test: 1.1368908317630717
Starting
Train: 1.1339068287411136, Test: 1.1412503017142028
Starting
Train: 1.1314456612709363, Test: 1.14011610949857


In [83]:
val_idx

array([      8,      21,      27, ..., 1199965, 1199992, 1199995])

In [21]:
stack_results = []
stack_models = []
for train_idx, val_idx in splitter.split(X_train):
    print("Starting")
    # Prepare data for LightGBM
    X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

    categorical_features = [col for col in X_train_fold.columns if X_train_fold[col].dtype == "category"]
    
    X_train_fold = X_train_fold.replace([pd.NA], np.nan)
    X_val_fold = X_val_fold.replace([pd.NA], np.nan)
    
    catboost_model = Pipeline([
        ("categorical_to_string", CatogoricalUnknownTransformer()),
        ("model",CatBoostRegressor(iterations=100,
                          early_stopping_rounds=10,
                          grow_policy = 'Depthwise',
                          depth=4,
                          cat_features=categorical_features,
                          random_state=42,
                          l2_leaf_reg = 1,
                          learning_rate=0.4,verbose=0))
    ])
    print("Fitting catboost - Start", datetime.now())
    catboost_model.fit(X_train_fold, y_train_fold)
    print("Fitting catboost - End", datetime.now())

    
    #linear_model_pipeline = Pipeline([
    #    ("preprocessing", build_preprocessing_pipeline(train_data=X_train_fold, imputation_method="simple", binning_method="opti-means", categorical_to_string=True)),
    #    ("model", LinearRegression())
    #])
    #print("Fitting linear model - Start", datetime.now())
    #linear_model_pipeline.fit(X_train_fold, y_train_fold)
    #print("Fitting linear model - End", datetime.now())
    
    """
    hist_pipeline = Pipeline([
        ("preprocessing", build_preprocessing_pipeline(train_data=X_train_fold, imputation_method="simple", categorical_to_string=True)),
        ("model", HistGradientBoostingRegressor())
    ])
    """
    
    xgboost = xgb.XGBRegressor(
            learning_rate=0.4290296903723802,
            n_estimators=187,
            max_depth=4,
            min_child_weight=1,
            subsample=0.8352284080073455,
            colsample_bytree=0.7100615955636335,
            gamma=2.8086205574716985,
            reg_lambda=2.9562811725383225,
            reg_alpha=9.51949783566794,
            enable_categorical=True,
            tree_method="hist",
            objective="reg:squarederror",
            device="cuda"
        )
    print("Fitting xgboost - Start", datetime.now())
    xgboost.fit(X_train_fold, y_train_fold)
    print("Fitting xgboost - End", datetime.now())

    stack_model = StackingRegressor(
        estimators=[
            #("linear_model", linear_model_pipeline),
            ("xgboost", xgboost),
            ("catboost", catboost_model)
        ],
        cv='prefit',
        #final_estimator=BayesianRidge(),
        final_estimator=xgb.XGBRegressor(enable_categorical=True, eta=0.1, colsample_bytree=0.9, n_estimators=100, max_depth=4, device="cuda", tree_method="hist"),
        n_jobs=-1
    )
    stack_models.append(stack_model)

    print("Fitting stack_model - Start", datetime.now())    
    stack_model.fit(X_train_fold, y_train_fold)
    print("Fitting stack_model - End", datetime.now())

    train_predictions = np.clip(stack_model.predict(X_train_fold), 20, 5000)
    val_predictions = np.clip(stack_model.predict(X_val_fold), 20, 5000)
    stack_results.append({
        "oof_predictions": {
            "index": val_idx,
            "predictions": val_predictions,
        },
        "test_predictions": {
            "predictions": np.clip(stack_model.predict(treated_test_dataset), 20, 5000),
        }
    })

    train_score = root_mean_squared_log_error(train_predictions, y_train_fold)
    val_score = root_mean_squared_log_error(val_predictions, y_val_fold)

    print(f"Train: {train_score}, Test: {val_score}")

Starting
Fitting catboost - Start 2024-12-30 01:08:08.145678
Fitting catboost - End 2024-12-30 01:08:18.428396
Fitting xgboost - Start 2024-12-30 01:08:18.428396
Fitting xgboost - End 2024-12-30 01:08:21.549552
Fitting stack_model - Start 2024-12-30 01:08:21.550553
Fitting stack_model - End 2024-12-30 01:08:22.608182
Train: 1.1203557160804973, Test: 1.1303386271811962
Starting
Fitting catboost - Start 2024-12-30 01:08:24.622183
Fitting catboost - End 2024-12-30 01:08:35.180649
Fitting xgboost - Start 2024-12-30 01:08:35.180649
Fitting xgboost - End 2024-12-30 01:08:38.181464
Fitting stack_model - Start 2024-12-30 01:08:38.181464
Fitting stack_model - End 2024-12-30 01:08:39.225506
Train: 1.1201931406560004, Test: 1.128188044562507
Starting
Fitting catboost - Start 2024-12-30 01:08:41.321513
Fitting catboost - End 2024-12-30 01:08:51.487303
Fitting xgboost - Start 2024-12-30 01:08:51.487303
Fitting xgboost - End 2024-12-30 01:08:54.582378
Fitting stack_model - Start 2024-12-30 01:08:54.

In [22]:
len(treated_test_dataset)

800000

In [23]:
oof = pd.concat([pd.DataFrame(stack_result['oof_predictions']) for stack_result in stack_results]).set_index('index').sort_index()
test_predictions = np.array([result['test_predictions']['predictions'] for result in stack_results]).mean(axis=0)

joblib.dump([oof,test_predictions],"cat_non_loged_local.pkl")


['cat_non_loged_local.pkl']